# Integrate melting curve of pure MgSiO3
* use direct clapeyron slope integration
* electronic contribution ignored due to bug in adiabatic calculation

In [ ]:
%run core.ipynb

analysis = load_analysis()
liq_eos_hybrid = analysis['liq_eos_hybrid']
liq_eos_S11 = analysis['liq_eos_S11']
liq_eos_dK09 = analysis['liq_eos_dK09']
sol_eos = analysis['sol_eos']

P_fus0 = analysis['P_fus0']
T_fus0 = analysis['T_fus0']
# dS_fus0 = analysis['dS_fus0']
# S_fus0 = analysis['S_fus0']


In [ ]:
PT_fus_Stx09 = pd.read_csv('data/MgSiO3-pure-melt-curve-Stixrude2009.csv')

## Calculate melting curve

In [ ]:
def melting_curve_deriv(P, TVlVs, liq_eos, sol_eos, dS_fus_kB=None):
    PV_ratio = xmeos.models.CONSTS['PV_ratio']
    T, Vliq, Vsol = TVlVs[0], TVlVs[1], TVlVs[2]
    # print('P', P, ', T', T, ', Vliq', Vliq, ', Vsol',Vsol)
    
    if dS_fus_kB is None:
        dS = liq_eos.entropy(Vliq, T) - sol_eos.entropy(Vsol, T)
    else:
        dS = dS_fus_kB*xmeos.models.CONSTS['kboltz']
        
    # print('   dS = ', dS)
    dV = Vliq - Vsol
     
    
    alpha_sol = sol_eos.thermal_exp(Vsol, T)
    alpha_liq = liq_eos.thermal_exp(Vliq, T)
    
    KT_sol = sol_eos.bulk_mod(Vsol, T)
    KT_liq = liq_eos.bulk_mod(Vliq, T)
    
    dTdP = dV/dS/PV_ratio
    # print('  dTdP = ', dTdP)
    dVdP_sol = Vsol*(alpha_sol*dTdP - 1.0/KT_sol)
    dVdP_liq = Vliq*(alpha_liq*dTdP - 1.0/KT_liq)
    TVV_deriv = np.hstack((dTdP, dVdP_liq, dVdP_sol))
    return TVV_deriv

In [ ]:

def calc_melting_curve(Pinit, Tinit, liq_eos, sol_eos, 
                       Pmax=150.0, N=101, Vinit=9.0, dS_fus_kB=None):
    Vinit_liq = volume(Pinit, Tinit, liq_eos, Vinit=Vinit)
    Vinit_sol = volume(Pinit, Tinit, sol_eos)
    
    # print(liq_eos.press(Vinit_liq, Tinit))
    # print(sol_eos.press(Vinit_sol, Tinit))
    
    deriv_fun = lambda P, TVlVs, liq_eos=liq_eos, sol_eos=sol_eos, dS_fus_kB=dS_fus_kB: (
        melting_curve_deriv(P, TVlVs, liq_eos, sol_eos, dS_fus_kB=dS_fus_kB) )
    
    # Vinit = volume(Pinit, Tinit, eos, Vinit=Vinit_guess)
    # Vinit = eos.volume(Pinit, Tinit, Vinit_guess)
    
    P_path = np.linspace(Pinit, Pmax, N)
    output = sp.integrate.solve_ivp(deriv_fun, P_path[[0,-1]] , 
                                    np.array([Tinit, Vinit_liq, Vinit_sol]), 
                                    t_eval=P_path, vectorized=False)
    T_fus_pos, V_fus_liq_pos, V_fus_sol_pos = (
        output['y'][0], output['y'][1], output['y'][2])
    
    
    # P_path_neg = np.linspace(P_fus0, Pmin, N)
    # output_neg = sp.integrate.solve_ivp(deriv_fun, P_path_neg[[0,-1]] , 
    #                                 np.array([Vinit, Tinit]), 
    #                                 t_eval=P_path_neg, vectorized=True)
    # V_ad_neg, T_ad_neg = output_neg['y'][0], output_neg['y'][1]
    # 
    # P_ad = np.hstack((P_path_neg[::-1][:-1], P_path))
    # V_ad = np.hstack((V_ad_neg[::-1][:-1], V_ad_pos))
    # T_ad = np.hstack((T_ad_neg[::-1][:-1], T_ad_pos))
    
    P_fus = np.hstack((P_path))
    T_fus = np.hstack((T_fus_pos))
    V_fus_liq = np.hstack((V_fus_liq_pos))
    V_fus_sol = np.hstack((V_fus_sol_pos))
    
    return P_fus, T_fus, V_fus_liq, V_fus_sol
    

In [ ]:
P_fus_S11, T_fus_S11, V_fus_liq_S11, V_fus_sol_S11 = calc_melting_curve(
    P_fus0, T_fus0, liq_eos_S11, sol_eos)
P_fus_dK09, T_fus_dK09, V_fus_liq_dK09, V_fus_sol_dK09 = calc_melting_curve(
    P_fus0, T_fus0, liq_eos_dK09, sol_eos)

dS_fus_kB = 1.5
P_fus_S11_fix, T_fus_S11_fix, V_fus_liq_S11_fix, V_fus_sol_S11_fix = calc_melting_curve(
    P_fus0, T_fus0, liq_eos_S11, sol_eos, dS_fus_kB=dS_fus_kB)
P_fus_dK09_fix, T_fus_dK09_fix, V_fus_liq_dK09_fix, V_fus_sol_dK09_fix = calc_melting_curve(
    P_fus0, T_fus0, liq_eos_dK09, sol_eos, dS_fus_kB=dS_fus_kB)


In [ ]:
# liq_eos_S11.press(V_fus_liq_S11, T_fus_S11)
# liq_eos_dK09.press(V_fus_liq_dK09, T_fus_dK09)
# sol_eos.press(V_fus_sol_S11, T_fus_S11)
# sol_eos.press(V_fus_sol_dK09, T_fus_dK09)

In [ ]:
P_ad_hybrid, V_ad_hybrid, T_ad_hybrid = integrate_adiabat(
    P_fus0, T_fus0, liq_eos_hybrid)

P_ad_dK09, V_ad_dK09, T_ad_dK09 = integrate_adiabat(
    P_fus0, T_fus0, liq_eos_dK09)

P_ad_S11, V_ad_S11, T_ad_S11 = integrate_adiabat(
    P_fus0, T_fus0, liq_eos_S11)

P_sol_ad, V_sol_ad, T_sol_ad = integrate_adiabat(
    P_fus0, T_fus0, sol_eos)

In [ ]:
plt.figure()
plt.plot(P_fus_dK09, T_fus_dK09, 'r-', lw=2, label='Melt Curve (dK09)')
plt.plot(P_fus_S11, T_fus_S11, 'b-', lw=2, label='Melt Curve (S11)')

plt.plot(P_fus_dK09_fix, T_fus_dK09_fix, 'r:', lw=2, label='Melt Curve (dK09-fix)')
plt.plot(P_fus_S11_fix, T_fus_S11_fix, 'b:', lw=2, label='Melt Curve (S11-fix)')

plt.plot(PT_fus_Stx09['P'], PT_fus_Stx09['T'], 'k:', label='Melt Curve (Stixrude)')


plt.plot(P_ad_hybrid, T_ad_hybrid, 'k-', lw=3, label='Adiabat (hybrid)')
plt.plot(P_ad_dK09, T_ad_dK09, 'r--', label='Adiabat (dK09)')
plt.plot(P_ad_S11, T_ad_S11, 'b--', label='Adiabat (S11)')
plt.plot(P_sol_ad, T_sol_ad, 'g-.', label='Adiabat (solid)')


plt.title('MgSiO3 Melting curve predictions')
plt.xlabel('Pressure  [ GPa ]')
plt.ylabel('Temperature  [ K ]')
plt.legend()

In [ ]:


S_fus_liq_S11 = liq_eos_S11.entropy(V_fus_liq_S11, T_fus_S11) 
S_fus_sol_S11 = sol_eos.entropy(V_fus_sol_S11, T_fus_S11) 
dS_fus_S11 = (S_fus_liq_S11-S_fus_sol_S11)/xmeos.models.CONSTS['kboltz']

S_fus_liq_dK09 = liq_eos_dK09.entropy(V_fus_liq_dK09, T_fus_dK09) 
S_fus_sol_dK09 = sol_eos.entropy(V_fus_sol_dK09, T_fus_dK09)
dS_fus_dK09 = (S_fus_liq_dK09-S_fus_sol_dK09)/xmeos.models.CONSTS['kboltz']



In [ ]:

plt.figure()

plt.plot(P_fus_dK09, dS_fus_dK09, 'r--')
plt.plot(P_fus_S11, dS_fus_S11, 'b--')

plt.xlabel('Pressure  [ GPa ]')
plt.ylabel('dS_fus  [ $k_B$ ]')


In [ ]:
Vad_dK09, Tad_dK09 = liq_compress_eos.adiabatic_path(Tinit, np.linspace(0,160,101))
Pad_dK09 = liq_compress_eos.press(Vad_dK09,Tad_dK09)
Sad_dK09 = liq_compress_eos.entropy(Vad_dK09, Tad_dK09)

Vad_S11, Tad_S11 = liq_thermal_eos.adiabatic_path(Tinit, np.linspace(0,150,101))
Pad_S11 = liq_thermal_eos.press(Vad_S11,Tad_S11)
Sad_S11 = liq_thermal_eos.entropy(Vad_S11, Tad_S11)

plt.figure()
plt.plot(Pad,Tad, 'r-', Pad_dK09, Tad_dK09, 'k--', Pad_S11, Tad_S11, 'b:')
plt.legend(['hybrid','dK09','S11'], loc='lower right')
plt.xlabel('Pressure [GPa]')
plt.ylabel('Temp. [K]')
plt.xlim(-2,140)
plt.ylim(2200,4000)

plt.savefig('figs/hybrid-ref-adiabat.png', dpi=450)
# plt.figure()
# plt.plot(Pad_dK09, Sad_dK09-np.mean(Sad_dK09) , 'k--', 
#          Pad_S11, Sad_S11 - np.mean(Sad_S11), 'b:')

In [ ]:

plt.figure()
plt.plot(Pad[:-1],(Tad[1:]-Tad[:-1])/(Pad[1:]-Pad[:-1]), 'r-',
         Pad_dK09[:-1], (Tad_dK09[1:]-Tad_dK09[:-1])/(Pad_dK09[1:]-Pad_dK09[:-1]), 'k--',
         Pad_S11[:-1], (Tad_S11[1:]-Tad_S11[:-1])/(Pad_S11[1:]-Pad_S11[:-1]), 'b:')
plt.legend(['hybrid','dK09','S11'], loc='upper right')
plt.xlabel('Pressure [GPa]')
plt.ylabel('dT/dP   [K/GPa]')
plt.xlim(-2,140)

plt.savefig('figs/hybrid-ref-adiabat-slope.png', dpi=450)


## Adjust liquid entropy to be realistic at reference point on melting curve

In [ ]:
def adjust_melting_entropy(P_fus0, T_fus0, sol_eos, liq_eos, dS_fus=1.5, Vliq_init=9.1):
    Vliq_fus0 = volume(P_fus0, T_fus0, liq_eos, Vinit=9)
    Vsol_fus0 = volume(P_fus0, T_fus0, sol_eos)
    
    # print(Vliq_fus0)
    # print(Vsol_fus0)

    # print(liq_eos.press(Vliq_fus0, T_fus0))
    # print(sol_eos.press(Vsol_fus0, T_fus0))

    liq_eos.set_param_values(0, param_names='S0')

    Sliq_fus0_init = liq_eos.entropy(Vliq_fus0, T_fus0)
    Ssol_fus0 = sol_eos.entropy(Vsol_fus0, T_fus0)

    S0 = Ssol_fus0 + dS_fus*CONSTS['kboltz'] - Sliq_fus0_init
    liq_eos.set_param_values(S0, param_names='S0')
    
    TOL = 1e-5
    S_error = (liq_eos.entropy(Vliq_fus0, T_fus0)
               - sol_eos.entropy(Vsol_fus0, T_fus0))/CONSTS['kboltz']-dS_fus
    assert np.abs(S_error)<TOL, (
        'fusion entropy does not match expectation... something wrong here.'
    )
    return S0




In [ ]:
P_fus0 = 25
T_fus0 = 2900

adjust_melting_entropy(P_fus0, T_fus0, mgpv_eos, liq_eos, dS_fus=1.5)
adjust_melting_entropy(P_fus0, T_fus0, mgpv_eos, liq_eos_dK09, dS_fus=1.5)

print('S0( S11) = ', liq_eos.get_param_values(param_names='S0')[0])
print('S0(dK09) = ', liq_eos_dK09.get_param_values(param_names='S0')[0])

In [ ]:
def dK09_funs():
    liq_funs = {}
    # liq_funs['entropy'] = lambda V, T: liq_eos.entropy(V, T)[0]
    liq_funs['entropy']  = liq_eos.entropy
    liq_funs['bulk_mod'] = liq_eos.bulk_mod
    liq_funs['alpha']    = liq_eos.thermal_exp
    liq_funs['press']    = liq_eos.press
    liq_funs['gibbs']    = lambda V, T: (
        liq_eos.internal_energy(V, T) 
        -T*liq_eos.entropy(V, T) 
        +liq_eos.press(V, T)*V/CONSTS['PV_ratio'])
    liq_funs['volume']   = lambda P, T: volume(P, T, liq_eos)
    
    sol_funs = {}
    sol_funs['entropy']  = mgpv_eos.entropy
    sol_funs['bulk_mod'] = mgpv_eos.bulk_mod
    sol_funs['alpha']    = mgpv_eos.thermal_exp
    sol_funs['press']    = mgpv_eos.press
    sol_funs['gibbs']    = lambda V, T: (
        mgpv_eos.internal_energy(V, T) 
        -T*mgpv_eos.entropy(V, T) 
        +mgpv_eos.press(V, T)*V/CONSTS['PV_ratio'])
    
    sol_funs['volume']   = lambda P, T: volume(P, T, mgpv_eos)
    
    return liq_funs, sol_funs

def S11_funs():
    return args

def hybrid_funs():
    return args

In [ ]:
liq_funs, sol_funs = dK09_funs()
print(liq_funs['entropy'](9, 2500))
print(sol_funs['entropy'](9, 2500))

print(liq_funs['bulk_mod'](9, 2500))
print(sol_funs['bulk_mod'](9, 2500))

print(liq_funs['alpha'](9, 2500))
print(sol_funs['alpha'](9, 2500))

print(liq_funs['volume'](9, 2500))
print(sol_funs['volume'](9, 2500))


print(liq_funs['gibbs'](9, 2500))
print(sol_funs['gibbs'](9, 2500))

In [ ]:
def melt_bound(P, T_Vl_Vs, liq_funs, sol_funs):
    T, Vl, Vs = T_Vl_Vs[:]
    
    Sl = liq_funs['entropy'](Vl, T)
    Ss = sol_funs['entropy'](Vs, T)
    
    dV = Vl-Vs
    dS = Sl-Ss
    
    KT_l = liq_funs['bulk_mod'](Vl, T)
    KT_s = sol_funs['bulk_mod'](Vs, T)
    
    alpha_l = liq_funs['alpha'](Vl, T)
    alpha_s = sol_funs['alpha'](Vs, T)
     
    dTdP = dV/dS/CONSTS['PV_ratio']
    dVldP = -Vl/KT_l + alpha_l*Vl*dTdP
    dVsdP = -Vs/KT_s + alpha_s*Vs*dTdP
    
    derivs = [dTdP, dVldP, dVsdP]
    return derivs





# solve_ivp(fun=lambda t, y: fun(t, y, *dK09_funs()), ...)

In [ ]:
P0 = 25
T0 = 2900
liq_dK09, sol_dK09 = dK09_funs()
V0l = liq_dK09['volume'](P0, T0)
V0s = sol_dK09['volume'](P0, T0)
dG0 = liq_dK09['gibbs'](V0l, T0)-sol_dK09['gibbs'](V0s, T0)
P_mod = np.linspace(P0, 136, 101)

output = sp.integrate.solve_ivp(lambda t, y: melt_bound(t, y, *(liq_dK09, sol_dK09)),
                       [P0,136], [T0, V0l, V0s], t_eval=P_mod, vectorized=True)
T_mod = output['y'][0]
Vl_mod = output['y'][1]
Vs_mod = output['y'][2]



In [ ]:
P0 = 25
T0 = 2900
liq_dK09, sol_dK09 = dK09_funs()
V0l = liq_dK09['volume'](P0, T0)
V0s = sol_dK09['volume'](P0, T0)
P_mod = np.linspace(P0, 12.5, 101)

output = sp.integrate.solve_ivp(lambda t, y: melt_bound(t, y, *(liq_dK09, sol_dK09)),
                                [P0,12.5], [T0, V0l, V0s], t_eval=P_mod, vectorized=True)
T_mod = output['y'][0]
Vl_mod = output['y'][1]
Vs_mod = output['y'][2]

In [ ]:
plt.figure()
plt.plot(P_mod, T_mod, 'k-')


In [ ]:
plt.figure()
plt.plot(PT_fus_Stx09['P'], PT_fus_Stx09['T'], 'k--')

In [ ]:
dP = P_mod[1]-P_mod[0]
dTdP_num = np.gradient(T_mod, dP)
dV_mod = Vl_mod-Vs_mod

Sl_mod = liq_dK09['entropy'](Vl_mod, T_mod)
Ss_mod = sol_dK09['entropy'](Vs_mod, T_mod)
dS_mod = Sl_mod-Ss_mod

dTdP_man = dV_mod/dS_mod/CONSTS['PV_ratio']

dG_mod = liq_dK09['gibbs'](Vl_mod, T_mod)-sol_dK09['gibbs'](Vs_mod, T_mod)

plt.figure()
plt.plot(P_mod, (liq_dK09['gibbs'](Vl_mod, T_mod)
                 -sol_dK09['gibbs'](Vs_mod, T_mod)-dG0), 'k-')

plt.figure()
plt.plot(P_mod, dTdP_num, 'k-', P_mod, dTdP_man, 'r--')

plt.figure()
plt.plot(P_mod, dTdP_num/dTdP_man - 1, 'r--')

plt.figure()
plt.plot(P_mod, dV_mod, 'k-')
plt.figure()
plt.plot(P_mod, dS_mod, 'k-')

In [ ]:

plt.figure()
# 

plt.plot(P_mod, P_mod - liq_dK09['press'](Vl_mod, T_mod), 'r--')
plt.plot(P_mod, P_mod - sol_dK09['press'](Vs_mod, T_mod), 'k--')


In [ ]:
def trace_phase_bound(Pinit, Tinit, Plim, phase1_eos, phase2_eos, 
                      Vinit1=None, Vinit2=None, N=100, full_output=False):
    
    Pbound = np.linspace(Pinit, Plim, N)
    Tbound = np.zeros(N)
    Tbound[0] = Tinit
    
    V1_bound = np.zeros(N)
    V2_bound = np.zeros(N)
    S1_bound = np.zeros(N)
    S2_bound = np.zeros(N)
    dV_bound = np.zeros(N)
    dS_bound = np.zeros(N)
    
    dP = Pbound[1]-Pbound[0]
    
    Vphase1_init = volume(Pinit, Tinit, phase1_eos, Vinit=Vinit1)
    Vphase2_init = volume(Pinit, Tinit, phase2_eos, Vinit=Vinit2)

    for ind, (P, T) in enumerate(zip(Pbound[:-1], Tbound[:-1])):
        Vphase1 = volume(P, T, phase1_eos, Vinit=Vphase1_init)
        Vphase2 = volume(P, T, phase2_eos, Vinit=Vphase2_init)
        Sphase1 = phase1_eos.entropy(Vphase1, T)[0]
        Sphase2 = phase2_eos.entropy(Vphase2, T)[0]
        
        dV = Vphase2-Vphase1
        dS = Sphase2 - Sphase1
        
        dV_bound[ind] = dV
        dS_bound[ind] = dS
        V1_bound[ind] = Vphase1
        V2_bound[ind] = Vphase2
        S1_bound[ind] = Sphase1
        S2_bound[ind] = Sphase2
        
        dTdP = dV/dS/CONSTS['PV_ratio']
        dT = dTdP*dP
        
        Pbound[ind+1] = P+dP
        Tbound[ind+1] = T+dT
        
        Vphase1_init = Vphase1
        Vphase2_init = Vphase2

    if full_output:
        output = {}
        output['V1'] = V1_bound
        output['V2'] = V2_bound
        output['S1'] = S1_bound
        output['S2'] = S2_bound
        output['dV'] = dV_bound
        output['dS'] = dS_bound
        return Pbound, Tbound, output
    else:
        return Pbound, Tbound
    return Pbound, Tbound

In [ ]:
def trace_hybrid_phase_bound(Pinit, Tinit, Plim, 
                             sol_eos, liq_thermal_eos, liq_compress_eos,
                             Vinit_sol=None, N=100, full_output=False):
    
    Pbound = np.linspace(Pinit, Plim, N)
    Tbound = np.zeros(N)
    Tbound[0] = Tinit
    
    dP = Pbound[1]-Pbound[0]
    
    Vsol_bound = np.zeros(N)
    Vliq_bound = np.zeros(N)
    dV_bound = np.zeros(N)
    dS_bound = np.zeros(N)
    
    Vsol_init = volume(Pinit, Tinit, sol_eos, Vinit=Vinit_sol)
    Vliq_init = hybrid_volume(Pinit, Tinit)

    for ind, (P, T) in enumerate(zip(Pbound[:-1], Tbound[:-1])):
        Vsol = volume(P, T, sol_eos, Vinit=Vsol_init)
        Vliq = hybrid_volume(P, T, Vinit=Vliq_init)
        
        Vsol_bound[ind] = Vsol
        Vliq_bound[ind] = Vliq
        
        
        dV = Vliq-Vsol
        dS = (hybrid_entropy(Vliq, T) 
              - sol_eos.entropy(Vsol, T))[0]
        
        dV_bound[ind] = dV
        dS_bound[ind] = dS
        
        dTdP = dV/dS/CONSTS['PV_ratio']
        dT = dTdP*dP
        
        Pbound[ind+1] = P+dP
        Tbound[ind+1] = T+dT
        
        Vsol_init = Vsol
        Vliq_init = Vliq
        

    if full_output:
        output = {}
        output['dV'] = dV_bound
        output['dS'] = dS_bound
        output['Vliq'] = Vliq_bound
        output['Vsol'] = Vsol_bound
        return Pbound, Tbound, output
    else:
        return Pbound, Tbound

In [ ]:
P_fus_LM, T_fus_LM =  trace_phase_bound(P_fus0, T_fus0, 136, mgpv_eos, liq_eos)
P_fus_dK09_LM, T_fus_dK09_LM =  trace_phase_bound(
    P_fus0, T_fus0, 136, mgpv_eos, liq_eos_dK09, Vinit2=9)
P_fus_hybrid_LM, T_fus_hybrid_LM =  trace_hybrid_phase_bound(
    P_fus0, T_fus0, 136, mgpv_eos, liq_eos, liq_eos_dK09)


In [ ]:
P_fus_UM, T_fus_UM =  trace_phase_bound(P_fus0, T_fus0, 12.5, mgpv_eos, liq_eos)
P_fus_dK09_UM, T_fus_dK09_UM =  trace_phase_bound(
    P_fus0, T_fus0, 12.5, mgpv_eos, liq_eos_dK09, Vinit2=9)
P_fus_hybrid_UM, T_fus_hybrid_UM =  trace_hybrid_phase_bound(
    P_fus0, T_fus0, 12.5, mgpv_eos, liq_eos, liq_eos_dK09)


In [ ]:
P_fus, T_fus =  trace_phase_bound(P_fus0, T_fus0, 136, mgpv_eos, liq_eos)
P_fus_dK09, T_fus_dK09, output_dK09 =  trace_phase_bound(P_fus0, T_fus0, 136, 
                                            mgpv_eos, liq_eos_dK09, Vinit2=9, 
                                                         full_output=True)


In [ ]:
P_fus_hybrid0, T_fus_hybrid0 =  trace_inconsistent_hybrid_phase_bound(
    P_fus0, T_fus0, 136, mgpv_eos, liq_eos, liq_eos_dK09, Vinit_liq_compress=9)


In [ ]:
P_fus_hybrid, T_fus_hybrid, output_hybrid =  trace_hybrid_phase_bound(
    P_fus0, T_fus0, 136, mgpv_eos, liq_eos, liq_eos_dK09, full_output=True)


In [ ]:
T_fus0

In [ ]:
P_fus_dK09, T_fus_dK09, output_dK09 =  trace_phase_bound(P_fus0, T_fus0, 12.5, 
                                            mgpv_eos, liq_eos_dK09, Vinit2=9, 
                                                         full_output=True)
P_fus_hybrid, T_fus_hybrid, output_hybrid =  trace_hybrid_phase_bound(
    P_fus0, T_fus0, 12.5, mgpv_eos, liq_eos, liq_eos_dK09, full_output=True)


In [ ]:
T0=1400
P_fus_dK09, T_fus_dK09, output_dK09 =  trace_phase_bound(10, T0, 25, 
                                            mgpv_eos, liq_eos_dK09, Vinit2=9, 
                                                         full_output=True)
P_fus_hybrid, T_fus_hybrid, output_hybrid =  trace_hybrid_phase_bound(
    10, T0, 25, mgpv_eos, liq_eos, liq_eos_dK09, full_output=True)


In [ ]:
plt.figure()
plt.plot(P_fus_dK09, output_dK09['V1'], 'k--',
        P_fus_dK09, output_dK09['V2'], 'r-')
plt.title('Vol')

plt.figure()
plt.plot(P_fus_dK09, output_dK09['S1'], 'k--',
        P_fus_dK09, output_dK09['S2'], 'r-')
plt.title('Entropy')


plt.figure()
plt.plot(P_fus_dK09, output_dK09['dS'], 'k--',
        P_fus_hybrid, output_hybrid['dS'], 'r-')
plt.title('dS')

plt.figure()
plt.plot(P_fus_dK09, output_dK09['dV'], 'k--',
        P_fus_hybrid, output_hybrid['dV'], 'r-')
plt.title('dV')

plt.figure()
plt.plot(P_fus_dK09, T_fus_dK09, 'k--',
        P_fus_hybrid, T_fus_hybrid, 'r-')
plt.title('Melt curve')



In [ ]:
plt.figure()

plt.plot(P_fus_hybrid, Vliq_hybrid,'k-',P_fus_hybrid,Vsol_hybrid,'b-')

In [ ]:
plt.figure()
plt.plot(P_fus, T_fus-2900, 'b-',
        P_fus_dK09, (T_fus_dK09-2900), 'r-',
        P_fus_hybrid0, T_fus_hybrid0-2900, 'k--',
        P_fus_hybrid, T_fus_hybrid-2900, 'g--')
plt.legend(['S11','dK09','crap-hybrid','hybrid'])


In [ ]:
plt.figure()
plt.plot(P_fus_hybrid,T_fus_hybrid)

In [ ]:
# V_hybrid = np.array([hybrid_volume(iP,iT)[0] for (iP,iT) in zip(P_fus_hybrid, T_fus_hybrid)])
S_liq = hybrid_entropy(Vliq_hybrid, T_fus_hybrid)
#V_sol = np.array([volume(iP,iT, mgpv_eos) for (iP,iT) in zip(P_fus_hybrid, T_fus_hybrid)])
# mgpv_eos.
S_sol = mgpv_eos.entropy(Vsol_hybrid, T_fus_hybrid)

In [ ]:
plt.figure()

plt.plot(P_fus_hybrid, S_liq,'k-',P_fus_hybrid,S_sol,'b-')

plt.figure()

plt.plot(P_fus_hybrid[1:], np.diff(S_liq)/np.diff(P_fus_hybrid),'b-')

In [ ]:
Vmod = np.linspace(6,15,100)

T_S0 = liq_eos.ref_temp_adiabat(Vmod)
T_S0_dK09 = liq_eos_dK09.ref_temp_adiabat(Vmod)

Pmod = liq_eos_dK09.press(Vmod, T_S0)

In [ ]:
plt.figure()
plt.plot(Vmod, T_S0, 'k-', Vmod, T_S0_dK09,'r--')

plt.figure()
plt.plot(Pmod, T_S0, 'k-', Pmod, T_S0_dK09,'r--')


In [ ]:

plt.figure()
plt.plot(P_fus, T_fus-2900, 'b-',
        P_fus_dK09, (T_fus_dK09-2900)/1.5, 'r--')
plt.legend(['S11','dK09 scaled'])
#plt.ylim(2500,5000)

# NOTE: Vinit value is off by 2% for some reason, needs addressing

In [ ]:
liq_thermal_eos = liq_eos
liq_compress_eos = liq_eos_dK09

Pinit = P_fus0
Tinit = T_fus0
Vinit = liq_compress_eos.volume(Pinit, Tinit)
dV = 0.01*Vinit

dPdT = liq_compress_eos.press(Vinit, Tinit+.5)-liq_compress_eos.press(Vinit, Tinit-.5)
Cv = liq_thermal_eos.heat_capacity(Vinit, Tinit)
gamma_hybrid = Vinit*dPdT/Cv/CONSTS['PV_ratio']

gamma_compress = liq_compress_eos.gamma(Vinit, Tinit)
gamma_thermal = liq_thermal_eos.gamma(Vinit, Tinit)

dTdV = -gamma_hybrid*Tinit/Vinit
print(dTdV)
dT = -dTdV*dV
T = Tinit+dT
print(T)
print(gamma_compress, gamma_thermal, gamma_hybrid)

In [ ]:
def trace_hybrid_adiabat(Pinit, Tinit, Plim, liq_thermal_eos, liq_compress_eos,
                         Vinit=None, N=100):
    
    Pad = np.zeros(N)
    Tad = np.zeros(N)
    gamma = np.zeroes(N)
    Tad[0] = Tinit
    Pad[0] = Pinit
    
    Vinit = volume(Pinit, Tinit, liq_compress_eos, Vinit=Vinit)
    dV = 0.01*Vinit

    for ind, (P, T) in enumerate(zip(Pbound[:-1], Tbound[:-1])):
        Pinit = liq_compress_eos.
        dPdT = (liq_compress_eos.press(Vinit, Tinit+.5)
                - liq_compress_eos.press(Vinit, Tinit-.5))
        Cv = liq_thermal_eos.heat_capacity(Vinit, Tinit)
        gamma_hybrid = Vinit*dPdT/Cv/CONSTS['PV_ratio']
        
        dTdV = -gamma_hybrid*Tinit/Vinit

        dT = -dTdV*dV
        
        
        gamma[ind] = gamma_hybrid
        Tad[ind+1] = Tinit+dT
        Pad[ind]
        
        
        
        Vsol = volume(P, T, sol_eos, Vinit=Vsol_init)
        Vliq = volume(P, T, liq_compress_eos, Vinit=Vliq_init)
        
        dV = Vliq-Vsol
        dS = (liq_thermal_eos.entropy(Vliq, T) 
              - sol_eos.entropy(Vsol, T))[0]
        
        dTdP = dV/dS/CONSTS['PV_ratio']
        dT = dTdP*dP
        
        Pbound[ind+1] = P+dP
        Tbound[ind+1] = T+dT
        
        Vsol_init = Vsol
        Vliq_init = Vliq

    return Pbound, Tbound

In [ ]:
liq_eos.heat_capacity(Vmod,T_S0)

In [ ]:
(liq_eos.internal_energy(Vmod,T_S0+.5)-liq_eos.internal_energy(Vmod,T_S0-.5))/liq_eos.heat_capacity(Vmod,T_S0)